# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 51 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.23151


extracting tarball to tmp_2505.23151...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.23342


extracting tarball to tmp_2505.23342...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2505.23342/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authorsandaffiliations' from 'tmp_2505.23342/authorsandaffiliations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'acronym' from 'tmp_2505.23342/acronym.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 148 bibliographic references in tmp_2505.23342/paper.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.23342-b31b1b.svg)](https://arxiv.org/abs/2505.23342) | **Euclid: Early Release Observations of ram-pressure stripping in the Perseus cluster. Detection of parsec scale star formation with in the low surface brightness stripped tails of UGC 2665 and MCG +07-07-070**  |
|| K. George, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-05-30*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Euclid is delivering optical and near-infrared imaging data over 14,000 deg$^2$ on the sky at spatial resolution and surface brightness levels that can be used to understand the morphological transformation of galaxies within groups and clusters. Using the Early Release Observations (ERO) of the Perseus cluster, we demonstrate the capability offered by Euclid in studying the nature of perturbations for galaxies in clusters. Filamentary structures are observed along the discs of two spiral galaxies with no extended diffuse emission expected from tidal interactions at surface brightness levels of $\sim$ $30\,{\rm mag}\,{\rm arcsec}^{-2}$. The detected features exhibit a good correspondence in morphology between optical and near-infrared wavelengths, with a surface brightness of $\sim$ $25\,{\rm mag}\,{\rm arcsec}^{-2}$, and the knots within the features have sizes of $\sim$ 100 pc, as observed through $I_E$ imaging. Using the Euclid, CFHT, UVIT, and LOFAR $144\,{\rm MHz}$ radio continuum observations, we conduct a detailed analysis to understand the origin of the detected features. We constructed the \textit{Euclid} $I_E-Y_E$, $Y_E-H_E$, and CFHT $u - r$, $g - i$ colour-colour plane and showed that these features contain recent star formation events, which are also indicated by their H$\alpha$ and NUV emissions. Euclid colours alone are insufficient for studying stellar population ages in unresolved star-forming regions, which require multi-wavelength optical imaging data. The morphological shape, orientation, and mean age of the stellar population, combined with the presence of extended radio continuum cometary tails can be consistently explained if these features have been formed during a recent ram-pressure stripping event. This result further confirms the exceptional qualities of Euclid in the study of galaxy evolution in dense environments.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.23151-b31b1b.svg)](https://arxiv.org/abs/2505.23151) | **A Be star-black hole binary with a wide orbit from LAMOST time-domain survey**  |
|| Q.-Y. An, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-05-30*|
|*Comments*| *76 pages, 29 figures, to be submitted*|
|**Abstract**|            Binary systems consisting of an early type star and a black hole (BH) are crucial for understanding various astrophysical phenomena, particularly the origins of detected gravitational wave sources. Be binary systems are expected to represent a key evolutionary stage in hosting BHs. However, while hundreds of Be X-ray binaries are known, the only confirmed BH candidate in a Be binary remains highly controversial. We report the discovery of ALS 8814, a Be star-BH binary with a moderately eccentric ($e = 0.23$) and wide orbit ($P = 176.6$ days), revealed by the radial velocity (RV) measurement of the visible Be star. Our analysis, combining flux-calibrated spectra in the Balmer discontinuity region and spectral template matching, yields a mass of $11.2^{+1.4}_{-1.2}$ $M_\odot$ for the Be star. The minimum mass of the unseen companion, assuming an edge-on inclination ($i = 90^{\circ}$), is $9.8\pm 0.7\,M_\odot$. We rule out the presence of non-degenerate companions in ALS 8814, indicating that it can only be a BH. This discovery represents a robust case of a Be-BH binary, identified purely through precise RV measurements from a single set of lines. The extremely low peculiar velocity of ALS 8814 suggests that the BH is formed via a direct core-collapse with a negligible natal kick, implying an almost perfect alignment between the Be star's spin and the orbital plane. In this context, the binary's inclination angle is estimated to be 22$^{\circ}$-49$^{\circ}$ by analyzing the shallow double-peaked profile of the H$\alpha$ emission line. This inclination range corresponds to a BH mass estimate between $15\,M_\odot$ and $58\,M_\odot$. As the only unambiguous Be-BH binary system known to date, ALS 8814 provides valuable constraints on the BH formation in a binary system with a high-mass companion.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.23342/./VIS_UGC2665_latest.png', 'tmp_2505.23342/./VIS_MCG+07-07-070_latest.png', 'tmp_2505.23342/./Euclid_perseus_color_composite_UGC2665.png']
copying  tmp_2505.23342/./VIS_UGC2665_latest.png to _build/html/
copying  tmp_2505.23342/./VIS_MCG+07-07-070_latest.png to _build/html/
copying  tmp_2505.23342/./Euclid_perseus_color_composite_UGC2665.png to _build/html/
exported in  _build/html/2505.23342.md
    + _build/html/tmp_2505.23342/./VIS_UGC2665_latest.png
    + _build/html/tmp_2505.23342/./VIS_MCG+07-07-070_latest.png
    + _build/html/tmp_2505.23342/./Euclid_perseus_color_composite_UGC2665.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# $\Euclid$: Early Release Observations of ram-pressure stripping in the Perseus cluster. Detection of parsec-scale star formation within the low surface brightness stripped tails of UGC 2665 and MCG +07-07-070$\thanks{This paper is published on behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.23342-b31b1b.svg)](https://arxiv.org/abs/2505.23342)<mark>Appeared on: 2025-05-30</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

K. George, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** _Euclid_ is delivering optical and near-infrared imaging data over 14 000 deg $^2$ on the sky at spatial resolution and surface brightness levels that can be used to understand the morphological transformation of galaxies within groups and clusters. Using the Early Release Observations (ERO) of the Perseus cluster, we demonstrate the capability offered by _Euclid_ in studying the nature of perturbations for galaxies in clusters. Filamentary structures are observed along the discs of two spiral galaxies, UGC 2665 and MCG +07-07-070, with no extended diffuse emission expected from tidal interactions at surface brightness levels of $\sim$ $30 {\rm mag} {\rm arcsec}^{-2}$ . The detected features exhibit a good correspondence in morphology between optical and near-infrared wavelengths, with a surface brightness of $\sim$ $25 {\rm mag} {\rm arcsec}^{-2}$ , and the knots within the features have sizes of $\sim$ 100 pc, as observed through $\IE$ imaging. Using the _Euclid_ , CFHT, UVIT, and LOFAR $144 {\rm MHz}$ radio continuum observations, we conduct a detailed analysis to understand the origin of the detected features. We constructed the _Euclid_ $\IE-\YE$ , $\YE-\HE$ , and CFHT $u - r$ , $g - i$ colour-colour plane and showed that these features contain recent star formation events, which are also indicated by their H $\alpha$ and NUV emissions. _Euclid_ colours alone are insufficient for studying stellar population ages in unresolved star-forming regions, which require multi-wavelength optical imaging data. There are features with red colours that can be explained by the presence of dust stripped along with the gas in these regions. The morphological shape, orientation, and mean age of the stellar population, combined with the presence of extended radio continuum cometary tails can be consistently explained if these features have been formed during a recent ram-pressure stripping event. This result further confirms the exceptional qualities of _Euclid_ in the study of galaxy evolution in dense environments.

</div>

<div id="div_fig1">

<img src="tmp_2505.23342/./VIS_UGC2665_latest.png" alt="Fig7" width="100%"/>

**Figure 7. -** Colour scale \IE image of galaxy UGC 2665 with the scaling set to highlight faint stripped features at the galaxy outskirts. The features that are likely part of the stripped tail are marked with cyan boxes. Coloured contours overlaid on the image are for NUV N245M imaging data of the galaxy in blue, H$\alpha$ in red and $144 {\rm MHz}$ radio continuum in black. The NUV and H$\alpha$ contour levels created for $2 \sigma$, $4 \sigma$ and $6 \sigma$ are shown. The contour levels created for $3 \sigma$, $6 \sigma$ and $12 \sigma$ from LOFAR image are shown. The direction of the stripping feature is marked with magenta-coloured arrows. White contours show isophote generated for surface brightness level of $30 {\rm mag} {\rm arcsec}^{-2}$. (*figure:fig3*)

</div>
<div id="div_fig2">

<img src="tmp_2505.23342/./VIS_MCG+07-07-070_latest.png" alt="Fig8" width="100%"/>

**Figure 8. -** Colour scale \IE image of galaxy MCG +07-07-070 with the scaling set to highlight faint stripped features at the galaxy outskirts. Details are same as in Fig. \ref{figure:fig3}. (*figure:fig4*)

</div>
<div id="div_fig3">

<img src="tmp_2505.23342/./Euclid_perseus_color_composite_UGC2665.png" alt="Fig5" width="100%"/>

**Figure 5. -** Colour-composite image of the UGC 2665 galaxy created by combining and assigning blue, green, and red colours for \IE, \YE, and \HE imaging data. The direction to the cluster centre is shown with a light magenta-coloured arrow. In the bottom right of the image, the blue patch is an artifact caused by dichroic ghost in \IE imaging. (*figure:fig1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.23342"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

506  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
